# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 26 2022 7:24PM,248349,21,412956565,"NBTY Global, Inc.",Released
1,Oct 26 2022 7:24PM,248599,21,412957302,"NBTY Global, Inc.",Released
2,Oct 26 2022 4:20PM,249470,16,8696381,Sartorius Bioprocess Solutions,Released
3,Oct 26 2022 4:16PM,249469,19,ADV80007090,"AdvaGen Pharma, Ltd",Released
4,Oct 26 2022 4:16PM,249469,19,ADV80007091,"AdvaGen Pharma, Ltd",Released
5,Oct 26 2022 4:16PM,249469,19,ADV80007092,"AdvaGen Pharma, Ltd",Released
6,Oct 26 2022 4:16PM,249469,19,ADV80007093,"AdvaGen Pharma, Ltd",Released
7,Oct 26 2022 4:16PM,249469,19,ADV80007094,"AdvaGen Pharma, Ltd",Released
8,Oct 26 2022 4:16PM,249469,19,ADV80007095,"AdvaGen Pharma, Ltd",Released
9,Oct 26 2022 4:16PM,249469,19,ADV80007096,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,249461,Released,1
10,249462,Released,5
11,249466,Released,1
12,249469,Released,12
13,249470,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Released
id,
249461,1
249462,5
249466,1
249469,12
249470,1


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Released
id,
248349,1
248599,1
249407,1
249408,1
249418,1


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,Released
id,
248349,1
248599,1
249407,1
249408,1
249418,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Released
0,248349,1
1,248599,1
2,249407,1
3,249408,1
4,249418,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,id,Released
0,248349,1
1,248599,1
2,249407,1
3,249408,1
4,249418,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc."
1,Oct 26 2022 7:24PM,248599,21,"NBTY Global, Inc."
2,Oct 26 2022 4:20PM,249470,16,Sartorius Bioprocess Solutions
3,Oct 26 2022 4:16PM,249469,19,"AdvaGen Pharma, Ltd"
15,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions
16,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation
21,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc."
22,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc."
23,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC"
24,Oct 26 2022 1:23PM,249451,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Released
0,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc.",1
1,Oct 26 2022 7:24PM,248599,21,"NBTY Global, Inc.",1
2,Oct 26 2022 4:20PM,249470,16,Sartorius Bioprocess Solutions,1
3,Oct 26 2022 4:16PM,249469,19,"AdvaGen Pharma, Ltd",12
4,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1
5,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5
6,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1
7,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc.",1
8,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC",1
9,Oct 26 2022 1:23PM,249451,21,"NBTY Global, Inc.",1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released
0,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc.",1
1,Oct 26 2022 7:24PM,248599,21,"NBTY Global, Inc.",1
2,Oct 26 2022 4:20PM,249470,16,Sartorius Bioprocess Solutions,1
3,Oct 26 2022 4:16PM,249469,19,"AdvaGen Pharma, Ltd",12
4,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1
5,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5
6,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1
7,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc.",1
8,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC",1
9,Oct 26 2022 1:23PM,249451,21,"NBTY Global, Inc.",1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc.",1
1,Oct 26 2022 7:24PM,248599,21,"NBTY Global, Inc.",1
2,Oct 26 2022 4:20PM,249470,16,Sartorius Bioprocess Solutions,1
3,Oct 26 2022 4:16PM,249469,19,"AdvaGen Pharma, Ltd",12
4,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released
0,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc.",1
1,Oct 26 2022 7:24PM,248599,21,"NBTY Global, Inc.",1
2,Oct 26 2022 4:20PM,249470,16,Sartorius Bioprocess Solutions,1
3,Oct 26 2022 4:16PM,249469,19,"AdvaGen Pharma, Ltd",12
4,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Oct 26 2022 7:24PM,248349,21,"NBTY Global, Inc.",1
1,Oct 26 2022 7:24PM,248599,21,"NBTY Global, Inc.",1
2,Oct 26 2022 4:20PM,249470,16,Sartorius Bioprocess Solutions,1
3,Oct 26 2022 4:16PM,249469,19,"AdvaGen Pharma, Ltd",12
4,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released
Warehouse,,
10,249462,5
12,498877,2
16,498936,2
19,498916,13
21,995806,4
22,748329,3


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released
0,10,249462,5
1,12,498877,2
2,16,498936,2
3,19,498916,13
4,21,995806,4
5,22,748329,3


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released
0,10,5
1,12,2
2,16,2
3,19,13
4,21,4
5,22,3


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,5
1,12,Released,2
2,16,Released,2
3,19,Released,13
4,21,Released,4


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,21,22
Status,,,,,,
Released,5,2,2,13,4,3


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,21,22
0,Released,5,2,2,13,4,3


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,21,22
0,Released,5,2,2,13,4,3


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()